# Practic examples of Systools Distribution Module

In [1]:
# importing
import sys
import os

sys.path.append(os.path.abspath(os.path.join('..')))

from main.systool import data, distribution
import pandas as pd

*distribution* is a module to perform the trip distribution, one of the steps of a Four Step Model on Transport Planning. The pce of code that we use was provided buy _Chetan Joshi_ and can be revised on the distribution.py file. 

Regarding the use of the availables functions it self, let's say that you have projections over dataset saved on a projections.csv and you also decided to use the regressions of the last step.

Mind that all those are examples with toy data, so the regressions may not make sense on real life.


In [2]:
df = data.open_file('examples_databases\input_distribuicao.txt')
df.head()

,ZONAS\tEMPREGOS\tPOP\tDOMICILIOS\tPEA
0,1\t42.438\t488.294\t132.446\t460.674
1,2\t229.825\t863.829\t705.817\t137.306
2,3\t41.812\t329.147\t122.317\t89.112
3,4\t156.163\t1010357\t829.559\t389.087
4,5\t60.082\t416.631\t348.208\t93.815


Stay alert! Always check what you are reading and investigate to resolve the problem.

In [3]:
df = data.open_file('examples_databases\input_distribuicao.txt', kwargs={'sep':'\t', 'decimal':'.'})
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119 entries, 0 to 118
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   ZONAS       119 non-null    int64  
 1   EMPREGOS    119 non-null    float64
 2   POP         119 non-null    float64
 3   DOMICILIOS  119 non-null    float64
 4   PEA         119 non-null    float64
dtypes: float64(4), int64(1)
memory usage: 4.8 KB


Now we need to generate our new ATRACTION and PRODUCTION vectors using our equations and projections for a future year

> regressionATRA = +0.314 * **EMPREGOS**

> regressionPROD = +0.019 * **PEA** +0.017 * **DOMICILIOS** +0.069 * **POP** +420.483

In [4]:
ATRA = 0.314 * df.EMPREGOS
PROD = 0.019 * df.PEA + 0.017 * df.DOMICILIOS + 0.069 * df.POP + 420.483

In [5]:
ATRA.shape

(119,)

Now that we have new vectors we can distribute them using any methodology available...

* **CalcFratar** : Calculates a Fratar/IPF on a seed matrix given row and column (P and A) totals
* **CalcSinglyConstrained** : Calculates a singly constrained trip distribution for given P/A vectors and a friction factor matrix
* **CalcDoublyConstrained** : Calculates a doubly constrained trip distribution for given P/A vectors and a friction factor matrix (P and A should be balanced before usage, if not then A is scaled to P)
* **CalcMultiFratar** : Applies fratar model to given set of trip matrices with multiple target production vectors and one attraction vector
* **CalcMultiDistribute** : Applies gravity model to a given set of frication matrices with multiple production vectors and one target attraction vector


You can always use help(function name) to understand better the paramters


In [6]:
help(distribution.calc_fratar)

Help on function calc_fratar in module main.systool.distribution:

calc_fratar(prod_a, attr_a, trips_1, max_iter=10, print_balance=False)
    Calculates fratar trip distribution
       prod_a = Production target as array
       attr_a = attr_action target as array
       trips_1 = Seed trip table for fratar
       max_iter (optional) = maximum iterations, default is 10
       Returns fratared trip table



In [7]:
df = data.open_file('examples_databases\input_geracao.xlsx', kwargs={'sheet_name':'Friction'})
df.head()

,ORIGEM,1,2,3,4,5,6,7,8,9,...,110,111,112,113,114,115,116,117,118,119
0,1,93,46,154,139,14,178,95,76,74,...,121,97,179,63,135,23,32,105,64,29
1,2,92,143,192,165,18,109,57,192,15,...,158,64,69,48,160,168,94,32,33,175
2,3,64,131,169,76,128,197,168,48,102,...,81,95,32,140,114,26,107,17,42,160
3,4,150,29,194,193,34,17,132,178,108,...,63,101,48,113,162,145,196,200,155,98
4,5,108,59,196,187,105,70,90,152,50,...,54,12,190,58,84,156,46,26,28,83


In [8]:
df.set_index('ORIGEM', inplace=True)

In [9]:
# attention! the parameters of distribution module are Numpy.Arrays not DataFrames
mtx = distribution.calc_fratar(PROD.values, ATRA.values, df.values)

You can save your matrix result and keep working on your python skills or modelling!

In [10]:
mtx

array([[2.21116490e+00, 7.96043351e+00, 3.93110639e+00, ...,
        1.43823812e-01, 1.08456199e+00, 3.89568391e-01],
       [2.05429384e+00, 2.32408216e+01, 4.60290338e+00, ...,
        4.11649912e-02, 5.25200221e-01, 2.20780298e+00],
       [1.40324706e+00, 2.09057699e+01, 3.97829298e+00, ...,
        2.14736762e-02, 6.56356333e-01, 1.98208228e+00],
       ...,
       [2.49008225e+00, 1.84638396e+01, 9.69120092e-01, ...,
        2.68978940e-02, 1.38656409e+00, 1.29258340e+00],
       [3.28834838e-01, 1.33133868e+01, 2.84644924e+00, ...,
        1.36162963e-01, 3.66213266e-01, 1.28892708e+00],
       [1.63786574e+00, 1.96699360e+01, 2.11016025e+00, ...,
        1.59466079e-01, 1.72773145e+00, 8.32852971e-01]])